In [1]:
from IPython.core.display import HTML
HTML("./custom.css")

<h1>How big are your banks?</h1>
<h2>The ones in the U.S. are <em>pretty</em> big!</h2>

<h3>Dr. David R. Pugh</h3>

<img src="./assets/img/pydata-logo-amsterdam-2016.png" alt="PyData Amsterdam">
<h4>Amsterdam, The Netherlands, 12-13 March 2016</h4>

<img src="../../assets/img/INET_Brand_Final-01.png" alt="Institute for New Economic Thinking" width=500>

<h2> Motivation </h2>

Benefits of banking sector concentration, such as increased profitability, economies of scale, and increased diversification accrue primarily to individual banks (and their shareholders). 

Costs of increased banking sector concentration, meanwhile, are largely social. These might include increased propensity to take risks due to moral hazard/adverse selection issues created by explicit deposit insurance arrangements as well as implicit bail-out arrangements for banks deemed "too big to fail," decreased competitiveness in the banking sector, etc.  

Characterizing the  behavior of the upper tail of the size distribution of banks is an important prerequisite to quantitatively assessing the potential costs and benefits from continued concentration of the U.S. banking sector.

<h3> I have a hypothesis! Zipf's Law... </h3>

Why? Strong empirical evidence from ???, ???, and ??? that distribution of firm sizes is well decribed by Zipf's Law. 

<h2> Data </h2>

<p>Data used in this study are taken from the <a href="http://www2.fdic.gov/sdi/index.asp">Statistics on Depository Institutions (SDI)</a> database maintained by the U.S. Federal Deposit Insurance Corporation (FDIC).<p>

SDI data set contains aggregate demographic and financial information about the U.S. banking sector, as well detailed <em>quarterly</em> data on individual bank (or bank holding company) balance sheets, income statements, performance ratios, etc., dating back to 1992.

<p>I look at six separate measures of bank size:</p>
<ul>
  <li><strong>Total assets (`asset`):</strong> The sum of all assets owned by the institution including cash, loans, securities, bank premises and other assets. This total does not include off-balance-sheet accounts.</li>
  <li><strong>Total loans (`lnlsnet`):</strong> Total loans and lease financing receivables minus unearned income and loan loss allowances.</li>
  <li><strong>Total liabilities (`liab`):</strong> Deposits and other borrowings, subordinated notes and debentures, limited-life preferred stock and related surplus, trading account liabilities and mortgage indebtedness.</li>
  <li><strong>Total deposits (`dep`):</strong> The sum of all deposits including demand deposits, money market deposits, other savings deposits, time deposits and deposits in foreign offices.</li>
  <li><strong>Total equity (`toteq`):</strong> Total equity capital on a consolidated basis.</li>
  <li><strong>Total employees (`numemp`):</strong> The number of full-time employees on the payroll of the bank and its subsidiaries at the end of the quarter.</li>
</ul>

Interested in the <em>shape</em> of the size distribution: I deflate and then re-scale each measure of bank size by dividing by banking sector totals relative to some base year, $T$...

$$  S_{i,t}^{norm} = \left(\frac{\sum_{j}S_{i, T}^{raw}}{\sum_{j}S_{j,t}^{raw}}\right)S_{i,t}^{raw}. $$

In case anyone asks...
<ul>
  <li>$S_{i,t}^{raw}$ denotes the raw size of bank $i$ in year $t$ based on one of the six size measures detailed above.</li>
  <li>$\sum_{j}S_{j,t}^{raw}$ is the banking sector total of some size measure in year $t$.</li>
  <li>$\sum_{j}S_{j,T}^{raw}$ is the banking sector total of the same size measure in the base year.</li>
</ul>

<h2> Exploratory data analysis </h2>

INSERT PLOT SHOWING TRENDS IN NUMBER OF BANKS!

<img src="./Market_shares_of_largest_bank.png" alt="???" width=400>

<h3>Significant agglomeration within the U.S. banking sector over the last 20+ years.</h3>

Key stats...
<ul>
  <li>Q1 1992: $\approx$ 14,000 banks in the data; largest bank (Citibank), controlled $\approx$ 3.5% of all U.S. banking assets.</li>
  <li>Q4 2011, $\approx$ 7,000 banks remain; largest bank (now JP Morgan-Chase), controlled $\approx$ 13% of all U.S. banking assets.</li>
  <li>"Jumps" in the share of assets and liabilities controlled by the largest U.S. bank all occurred because of mergers between large banks:
    <ul>
      <li>1999: Nations Bank (largest U.S. bank in 1999), merged with Bank of America.</li> 
      <li>2004: JPMorgan-Chase (the largest bank in 2003), purchased Bank One (6th largest bank).</li>
      <li>2008: JPMorgan-Chase acquired both Bear Stearns and Washington Mutual</li>
    </ul>
  </li>
  <li>"Jumps" in market shares also hint at another important stylized fact: there seems to be no preferred scale for bank mergers.</li>
</ul>

This figure shows the fraction of total U.S. banking assets, loans, liabilities, deposits, equity, and employees controlled by the single largest bank from 1992 to 2011.

<ul>
  <li>Monetary figures are first deflated and then re-scaled by dividing through by banking sector totals relative to 2011.</li>
</ul>

<img src="./Measures_of_bank_size.png" alt="???" width=400>

Extent of agglomeration within the banking sector even more dramatic when one examines the market shares held by the ten largest U.S. banks. 

<ul>
  <li>Market share of 10 largest banks has more than doubled from less than 20% to roughly 50%.</li>
</ul>


<h3>...distributions are heavily skewed!</h3>

<img src="./Normed_Assets_density_1992_2011.png" alt="???" width=400>

<img src="./Survival_functions_normed_loans_1992-2011.png" alt="Upper ECDF for loans and leases (net)" width=400>

<h2>Methodology</h2>

Follow the methodology advocated by <a href="http://arxiv.org/abs/0706.1062">Clauset, Shalizi, and Newman (SIAM, 2009)...</a>

<h3> Don't use OLS, use maximum likelihood!</h3>

For a given value of $x_{min}$, the maximum likelihood estimator for the scaling exponent is

$$ \hat{\alpha} = 1 +  n\left[\sum_{i=1}^{n}\mathrm{ln}\ \left(\frac{x_{i}}{x_{min}}\right)\right]^{-1}. $$

But this maximum likelihood estimator holds only for a <em>given</em> value of $x_{min}$...
<ul>
  <li>\cite{clauset2009power} suggest estimating $x_{min}$ from the data by choosing a value $\hat{x}_{min}$ that minimizes the "distance" between the empirical distribution of the observed data and the best-fit power law model above $\hat{x}_{min}$.</li>
</ul>

In case anyone asks the \cite{clauset2009power} procedure for choosing $x_{min}$ works as follows...
<ol>
  <li>Choose a set of candidate threshold parameters, $\Theta$ from the observed data.</li>
  <li>For each $x_{min}^{c} \in \Theta$ obtain an estimate of the scaling exponent $\hat{\alpha}^{c}$ using maximum likelihood and then calculates the Kolmogorov-Smirnov (KS) distance between the empirical distribution of the data above $x_{min}^c$ and the theoretical distribution of a power law with parameters $\hat{\alpha}^{c}$ and $x_{min}^c$.</li>
  <li>Optimal choice for the threshold parameter, $x_{min}$, is the $x_{min}^{*} \in \Theta$ which minimizes the KS distance between the observed data above $x_{min}^{*}$ and the theoretical power law distribution with scaling exponent, $\hat{\alpha}^{*}$.</li>
</ol>

<p>In order to get estimates of parameter uncertainty that accurately take into account the flexibility introduced by the joint estimation of $\alpha$ and $x_{min}$, standard errors and confidence intervals for the parameter estimates are estimated using a basic non-parametric bootstrap procedure detailed in \cite{davison1997bootstrap}.</p>

<h3>Don't forget to assess goodness of fit for the power law model?</h3>

<h3>Don't forget to test alternative hypotheses!</h3>

Not enough just to fit a power law distrubtion to the data and write a paper...there might be distributions that are an even better fit!

I consider three common alternative distributions for heavy-tailed data...
<ul>
  <li>log-normal</li>
  <li>stretched-exponential (i.e., Weibull)</li>
  <li>power law with an exponential cut-off</li>
</ul>
...and one thin-tailed alternative: the exponential distribution.

<p>I use a <a href="https://en.wikipedia.org/wiki/Vuong%27s_closeness_test">Vuong likelihood ratio test</a> implemented in two steps to test off the power law model against various alternatives...</p>
<ol>
  <li> Consider the two-sided null hypothesis that each of the power law and the alternative are equally far from the "true distribution" against a general alternative.  If I reject this two-sided null hypothesis, then I conclude that one of the power law or the alternative is preferred (given the data).</li>
  <li> Second I consider a one-sided null hypothesis of a power law against the alternative distribution directly.</li>
</ol>
  
<p>A rejection of the two-sided null hypothesis, followed by a failure to reject the one-sided null hypothesis of a power law implies the data are sufficient to distinguish between the power law and the considered alternative, and the the power law model if preferred.</p>

A major advantage of using the Vuong likelihood ratio test over the classic likelihood ratio test is that the former will alert the researcher to the possibility that the data are not sufficient to favor either the power law or the alternative model.

<h2>Results</h2>

<h3>Parameter estimates</h3>

<img src="./Estimated_Scaling_Exponents.png" alt="Estimated scaling exponents">

<h3>Tail of the size distribution is <em>too heavy</em> to be consistent with Zipf's Law</h3>
<p>Figure reports estimation results for the scaling exponent $\alpha$...<p>
<ul>
  <li>Tail of the size distribution is so heavy that the mean of the best-fitting power law model is undefined! There is no such thing as an "average" sized bank in the U.S.!</li>
  <li>Only measure of bank size broadly consistent with Zipf's Law is number of employees.</li>
  <li>When either `asset`, `lnlsnet`, or `liab` are used as the measure of bank size $\hat{\alpha}$ is roughly constant across time.</li>
  <li>When `dep` are used as the measure of bank size the $\hat{\alpha}$ is slightly larger (though still significantly less than $\alpha=2$).</li>
  <li> Only measure of size for which there is a clear trend in $\hat{\alpha}$ is `eqtot`. The negative trend indicates that U.S banking sector equity holdings are becoming increasingly concentrated over time.
</ul>

<h3>Goodness of fit tests</h3>

<h3>Likelihood Ratio tests</h3>

<img src="./Asset_distributions_with_power_law_vs_alternatives(2011).png" alt="Estimated scaling exponents">

Summary of likelihood ratio testing results...
<ul>
  <li>Both the exponential distribution and the stretched exponential distribution are inferior to the power law model.</li>
  <li>Given the data it is generally not possible to distinguish between the power law and the log-normal alternative.</li>
  <li>Finally, while the power law distribution with an exponential cut-off is preferred over the pure power law model in the early 1990's, this distribution becomes less plausible over time.</li>
</ul>  

<h2> Conclusions </h2>

<p>Statistically significant departures from Zipf's Law for most measures of bank size in most years...</p>
<ul>
  <li>$\hat{\alpha} \approx 1.9 < 2$ indicates that U.S. the banking sector is too concentrated to be consistent with the predictions of Zipf's Law.</li>
  <li>Goodness-of-fit tests document statistically significant deviations from a pure power law model, it is not clear whether or not the observed deviations are economicially significant.</li>
  <li>Likelihood ratio testing results indicate that the power law model out-performs other distributions commonly used to model heavy-tailed data.</li>
</ul>

<strong>Conclusion:</strong> Power law distribution with $\hat{\alpha} \approx 1.9$ is a pretty good model for the upper tail of the size distribution of U.S. banks.

<h3>Implications for empirical and theoretical business cycle research?</h3> 

\cite{gabaix2011granular} posits that, if upper tail of the firm size distribution is sufficiently heavy, significant variation in aggregate macroeconomic time series could be explained by idiosyncratic shocks to individual firms.

Presumbably there is a "granularity hypothesis" for the banking sector where idiosyncratic shocks to systematically important financial institutions (SIFIs) could also explain some variation in aggregate macroeconomic time series.

<h2> Thanks! </h2>

<h2> Questions? </h2>